In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/ray__/ssd/BERT/")
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
sys.path.append("../")
from gpt_feat_utils import GPT_Inference

gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")
#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/ether/", device="cpu")

# with open('../topic_testing/cullen_test.json','rb') as f:
#     request = json.load(f)
#     if isinstance(request, str):
#         request = json.loads(request)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [4]:
import pickle
import numpy as np
import networkx as nx

# S.E
#ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl","rb"))
#sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/se_sent_dict_v3.pkl", "rb"))
#com_map = pickle.load(open("/home/ray__/ssd/minds/se/tests/com_map.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/se/se_ent_graph_wcosine_pruned_v3.gpkl")

# Ether
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether/ether_entity_feats_grp.pkl","rb"))
#sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_sent_dict_v1.pkl", "rb"))
label_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_single_label_dict_v1.pkl","rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/ether/com_map_h.pkl", "rb"))

# Ether_se

# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether_se/se_ether_feats_non_updated_allether.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ether_se/com_map_h.pkl", "rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_single_label_dict_v1.pkl","rb"))

# # A.Ient
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ai/ai_entity.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ai/com_map_ai.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/ai/ai_pruned_entity_wfv.gpkl")


# common_entities = ent_fv_full.keys() & com_map.keys()
# ent_fv = {}
# for ent in common_entities:
#     ent_fv[ent] = ent_fv_full[ent]

common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    if ent in sent_dict.keys() and sent_dict[ent]=='PER':
        continue
    ent_fv[ent] = ent_fv_full[ent]

In [23]:
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text
import numpy as np
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from main import handler

sent_to_fv = pickle.load(open("../notebooks/sent_to_fv.pkl", "rb"))
sent_to_entity = pickle.load(open("../notebooks/sent_to_entity.pkl", "rb"))
    
def get_entities(groupobj):
    seg_list = " ".join([seg['originalText'] for seg in groupobj])
    sent_list = preprocess_text(seg_list)
    sent_fv = [gpt_model.get_text_feats(s) for s in sent_list]
    group_ent_list = []
    
    for query_fv in sent_fv:
        #print (len(query_fv))
        #print(np.reshape(query_fv, (1,-1)).shape)
        res_sim = cdist(np.reshape(query_fv, (1,-1)), np.stack(sent_to_fv.values(), axis=0), 'cosine')

        #res_sim_indexed = []
        #for index, score in enumerate(res_sim[0]):
        #    res_sim_indexed.append((index,score))

        #top_3 = sorted(res_sim_indexed, key= lambda kv:kv[1], reverse=False)[:3]
        index = np.argmin(res_sim)
        group_ent_list.append(sent_to_entity[list(sent_to_fv.keys())[index]])
    return group_ent_list

def generate_gs(set_list):
    com_freq = {}
    for file in set_list:
        req = json.load(open(file, "r"))
        if isinstance(req, str):
            request = json.loads(req)["body"]
        else:
            request = req["body"]
            
        group = json.loads(handler(req, None)['body'])['group']
        group_ent = {}
        for  groupid, groupobj in group.items():
            group_ent[groupid] = get_entities(groupobj)
            
        for groupid, ent_list in group_ent.items():
            for ent in ent_list:
                if ent in com_map.keys():
                    if com_map[ent] in com_freq.keys():
                        com_freq[com_map[ent]] += 1
                    else:
                        com_freq[com_map[ent]] = 1
            #print ("\n\n\n", segments_map[segid]["originalText"], "\n\n Entities picked: ", [i for i,j in sorted_score][:10], "\n\n community mapped: ",  [com_map[i] for i in [i for i,j in sorted_score][:10]])
    return com_freq

In [24]:
import os
dataset = os.listdir("../golden_dataset/ether/")

ranked = generate_gs(["../golden_dataset/ether/"+i for i in dataset if i!=".ipynb_checkpoints"])

Using  01DAAQY88QZB19JQZ5PRJFR76Y  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 93, "ts": "2020-01-21T10:53:03.466879Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-01-21T10:53:03.468544Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-01-21T10:53:08.245980Z", "msg": "Request Sent"}
('I just done with some pocs one thing which I wanted to figure it out like why it was happening.', '2020-01-14T05:50:06Z', '65bb83952fb54409a4bb59bb707f1375', '0f73d7e8-587f-4546-9180-99e18202242d') ('But one more effort we had was the mixing and then I was doing the mixing yesterday.', '2020-01-14T05:50:35Z', '65bb83952fb54409a4bb59bb707f1375', 'a245e6a2-5b8f-4d60-a4b8-b78fff2aa084')
('I just done with some pocs 

{"level": "info", "filename": "grouper_segments.py", "lineno": 1039, "module": "grouper_segments", "edges before prunning": 473, "edges after prunning": 473, "modularity": 0.6731901658807248, "ts": "2020-01-21T10:53:09.780900Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
I just done with some pocs one thing which I wanted to figure it out like why it was happening.
I frame and why it is not in the with the web and if I hosted our entire lives, it is a mate on the iframe whether I could able to hear the sounds or not.
But one more effort we had was the mixing and then I was doing the mixing yesterday.
I was booking on the mixing part in that like we do not have to do much work by because it seemed it always gives the updated strength initially only once we have to do it because since I was Computing again, and again, it was giving me Cool.
And also I get the transcript the problem now I am facing is if I have like attachment my microphone and if I speak I am abl

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10)]
[[["I mean, they don't have to listen for the renegotiation on soft because the streams which helps it gives in the initial node will be automatically updated. Like if we hear the blocks which they send the like if we transform the stream to blob and if you hear the jobs we get the entire conversation. "], '2020-01-14T05:53:23Z', '65bb83952fb54409a4bb59bb707f1375', '2cc019f9-0290-4bc6-afe5-156b8cebd1f4'], [["Aztecs added we need to compute but then if some tracks are removed or if someone mute or unmute we don't have to compute again. "], '2020-01-14T05:53:54Z', '65bb83952fb54409a4bb59bb707f1375', '898b0ff5-3ad5-46a0-9647-45533717e703'], [["Don't try and deploy it on staging to recorder and you can test like how it's coming actually in you haven't deployed on recorder yet, right? "], '2020-01-14T05:54:24Z', '0bbbfe84c66145af8d0ffcd5258bba38', 'aeb6ad04-ee41-4cab-b32b-206205c23c9e'], [["Yeah, so l

/home/ray__/.pipenv/graph/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


Using  01DAAYHEKY5F4E02QVRJPTFTXV  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 152, "ts": "2020-01-21T10:53:19.714311Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-01-21T10:53:19.716152Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-01-21T10:53:27.447528Z", "msg": "Request Sent"}
('If we plant again like then staging to I will have to deploy the branch basically Master which we are planning to deploy on products.', '2020-01-08T06:33:00Z', '0bbbfe84c66145af8d0ffcd5258bba38', '04660786-3748-4649-8ae7-ef97fd4c54b7') ('After the gold one meeting you start depending on the branches that want to push to stay into and once placed on me and then you can go back and whatever we can do play whatever

...

Using  01DAAYHEKY5F4E02QVRJPTFTXV  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 77, "ts": "2020-01-21T10:54:18.351404Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-01-21T10:54:18.356223Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-01-21T10:54:22.394584Z", "msg": "Request Sent"}
('So the way of thinking about it was if you see the chapter marker for this call you have hey guys, and hey Frenchy and all that stuff, right?', '2020-01-03T06:41:23Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', '0b626c6b-f4c2-417e-bb00-c3e81a2632a8') ('So I have been looking at those at least in the ones that the recommendations are pretty good, especially if you remove your point folks are not part of the call and if w

{"level": "info", "filename": "grouper_segments.py", "lineno": 1039, "module": "grouper_segments", "edges before prunning": 260, "edges after prunning": 260, "modularity": 0.7547149349127698, "ts": "2020-01-21T10:54:23.425191Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
So the way of thinking about it was if you see the chapter marker for this call you have hey guys, and hey Frenchy and all that stuff, right?
So I have been looking at those at least in the ones that the recommendations are pretty good, especially if you remove your point folks are not part of the call and if we look at confidence of XnumberX a greater or something like that results seem to be pretty good especially forecast.
I think it is going to be very useful because if we can actually mark it around the future that will cause because past is only one person.
So almost everyone else is not going to be in the call and so suppose I leave a message about whatever right like I point out somethi

Using  01DAAQY88QZB19JQZ5PRJFR76Y  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 86, "ts": "2020-01-21T10:54:36.477655Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-01-21T10:54:36.478861Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-01-21T10:54:40.886572Z", "msg": "Request Sent"}
('Okay, so maybe just spend some time so that we are not wasting time trying to identify test cases station, right you and Franklin sit down and decide what it is that we want test.', '2020-01-16T06:19:03Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', '00aca236-2caa-42e9-a9b6-29b8d3b849e1') ('So yeah, once once franklyn starts to test it I can deploy.', '2020-01-16T06:17:52Z', '75bdf310110b4b8fab88b16fafce920e', '0bae3b28-

{"level": "info", "filename": "grouper_segments.py", "lineno": 1039, "module": "grouper_segments", "edges before prunning": 374, "edges after prunning": 374, "modularity": 0.7354487123002627, "ts": "2020-01-21T10:54:42.174372Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
Okay, so maybe just spend some time so that we are not wasting time trying to identify test cases station, right you and Franklin sit down and decide what it is that we want test.
We do not want the end to a scenario where we wait for the summaries and all we just want to verify whether the body is joining at the scheduled time or not.
So with that shank is going to put that into the FIFA service and then see how how it is performing with respect to filtration.
So in that case what happens is they come as a they come as a usual usable knows that we cannot filter with the current approach yet.
But this has happened this is this is rare, but this is one flaw that I have seen with the Also has bee



====================Group Cluster=========================
[[["Oh, no it is because they promote more hits on the website and it was like because of the headset purposely like they were few memory issues and that's why like, uh, we were playing at us. So that's right. "], '2020-01-16T06:10:06Z', '0bbbfe84c66145af8d0ffcd5258bba38', 'e36aa01f-85ed-4882-97ee-0c7a4f4e585a'], [['You know, like how many like what would be the typical memory requirement, right? It should be able to do reasonably. Well well with like all like what is the memory we have allocated right now so long? '], '2020-01-16T06:10:33Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', '4005b88c-bbfe-42ec-b6a6-d88d9cc9f4ed'], [['It was just like 128mb for PHP. That was the limit by default which was said earlier. Okay. All right. Well, we have changed it to Alexis the easy to container has a 3gp like limit 40 GB memory if we have made PHP Summer Olympic 2.5 GB '], '2020-01-16T06:10:47Z', '0bbbfe84c66145af8d0ffcd5258bba38', '8591da0d-

In [25]:
ranked_sorted = sorted(ranked.items(), key=lambda kv:kv[1], reverse=True)

clusters = {}
temp = []
prev_com = 0
for index,(word,cluster) in enumerate(sorted(com_map.items(), key=lambda kv:kv[1])):
    if prev_com==cluster:
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp
    else:
        clusters[prev_com] = temp
        temp = []
        prev_com = cluster
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp

In [26]:
for com, freq in ranked_sorted:
    print ("\n\nCommunity: ", com, "  Freq: ", freq ,"\n")
    print (clusters[com])



Community:  13   Freq:  43 

['Ios', 'Web']


Community:  42   Freq:  22 

['Json', 'Uuid', 'Uuids']


Community:  14   Freq:  18 

['Internet', 'Google']


Community:  65   Freq:  17 

['Ether Engine', 'Entity Graph', 'Entity']


Community:  55   Freq:  15 

['Skull']


Community:  21   Freq:  11 

['Epa', 'Slack', 'Admin', 'Psyche']


Community:  3   Freq:  10 

['Apa', 'Apis', 'Api', 'Sigmoid']


Community:  15   Freq:  10 

['Zoom', 'Ios App', 'App', 'App Store']


Community:  26   Freq:  6 

['Pakistan']


Community:  5   Freq:  6 

['Cdn', 'Aws', 'S3']


Community:  39   Freq:  5 

['Controller', 'Event', 'Context']


Community:  36   Freq:  5 

['Websockets', 'Websocket']


Community:  7   Freq:  5 

['Xcode', 'Bill', 'Beta', 'App Center', 'Native']


Community:  12   Freq:  5 

['Android', 'Chrome']


Community:  45   Freq:  5 

['Ml']


Community:  4   Freq:  5 

['Lambda', 'Lambdas', 'Cloudwatch']


Community:  10   Freq:  3 

['Electron', 'React', 'Iframe', 'Electron App',

In [27]:
com_map_selected = {}
for index, (com, freq) in enumerate(ranked_sorted):
    com_map_selected[com] = index

In [28]:
pickle.dump(com_map_selected, open("/home/ray__/ssd/minds/ether/ranked_com_h.pkl", "wb"))

In [30]:
com_map_selected[com_map['Ios']]

0

In [31]:
com_map['Ios']

13